In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1.1


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss(num_neg=4,margin=0.1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.DynamicClip.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7945.12it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5688.24it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3417428.87it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11195.83it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5807.18it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1063821.03it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 11181.14it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fbf31d68b70>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fbf2f64f710>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    batch_size=100,
    resample=True,
    sort=False,
    shuffle=True
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=160,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.DynamicClip.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre'
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.DynamicClip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'max'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_topk'] = 10
model.params['right_topk'] = 5

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DynamicClip(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (attention): DynamicClipAttention()
  (conv_activation): ReLU()
  (hidden_activation): Tanh()
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (W): Linear(in_features=7040, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Trainable params:  3984233


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-8)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-11 Loss-0.087]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.475 - normalized_discounted_cumulative_gain@5(0.0): 0.5473 - mean_average_precision(0.0): 0.5135



[Iter-22 Loss-0.064]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5662 - normalized_discounted_cumulative_gain@5(0.0): 0.6166 - mean_average_precision(0.0): 0.579



[Iter-33 Loss-0.045]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6166 - normalized_discounted_cumulative_gain@5(0.0): 0.6701 - mean_average_precision(0.0): 0.6276



[Iter-44 Loss-0.033]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6393 - normalized_discounted_cumulative_gain@5(0.0): 0.6846 - mean_average_precision(0.0): 0.6456



[Iter-55 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6446 - normalized_discounted_cumulative_gain@5(0.0): 0.7029 - mean_average_precision(0.0): 0.6552



[Iter-66 Loss-0.012]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.656 - normalized_discounted_cumulative_gain@5(0.0): 0.7088 - mean_average_precision(0.0): 0.6624



[Iter-77 Loss-0.008]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6651 - normalized_discounted_cumulative_gain@5(0.0): 0.7143 - mean_average_precision(0.0): 0.6712



[Iter-88 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6659 - normalized_discounted_cumulative_gain@5(0.0): 0.7168 - mean_average_precision(0.0): 0.675



[Iter-99 Loss-0.003]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6539 - normalized_discounted_cumulative_gain@5(0.0): 0.7044 - mean_average_precision(0.0): 0.6649



[Iter-110 Loss-0.003]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6633 - normalized_discounted_cumulative_gain@5(0.0): 0.7108 - mean_average_precision(0.0): 0.6693

Cost time: 94.21824598312378s


In [13]:
model = mz.models.DynamicClip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'threshold'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_threshold'] = 0.08
model.params['right_threshold'] = 0.08

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DynamicClip(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (attention): DynamicClipAttention()
  (conv_activation): ReLU()
  (hidden_activation): Tanh()
  (conv1d_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (W): Linear(in_features=7040, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Trainable params:  3984233


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-8)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [15]:
trainer.run()

[Iter-11 Loss-0.083]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4823 - normalized_discounted_cumulative_gain@5(0.0): 0.5478 - mean_average_precision(0.0): 0.5121



[Iter-22 Loss-0.069]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5784 - normalized_discounted_cumulative_gain@5(0.0): 0.6319 - mean_average_precision(0.0): 0.5964



[Iter-33 Loss-0.047]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6131 - normalized_discounted_cumulative_gain@5(0.0): 0.66 - mean_average_precision(0.0): 0.6273



[Iter-44 Loss-0.035]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6456 - normalized_discounted_cumulative_gain@5(0.0): 0.6937 - mean_average_precision(0.0): 0.652



[Iter-55 Loss-0.020]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6576 - normalized_discounted_cumulative_gain@5(0.0): 0.7054 - mean_average_precision(0.0): 0.6663



[Iter-66 Loss-0.013]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6491 - normalized_discounted_cumulative_gain@5(0.0): 0.6996 - mean_average_precision(0.0): 0.6548



[Iter-77 Loss-0.011]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.671 - normalized_discounted_cumulative_gain@5(0.0): 0.7112 - mean_average_precision(0.0): 0.6688



[Iter-88 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6624 - normalized_discounted_cumulative_gain@5(0.0): 0.7102 - mean_average_precision(0.0): 0.6691



[Iter-99 Loss-0.004]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.668 - normalized_discounted_cumulative_gain@5(0.0): 0.7057 - mean_average_precision(0.0): 0.672



[Iter-110 Loss-0.002]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6797 - normalized_discounted_cumulative_gain@5(0.0): 0.7122 - mean_average_precision(0.0): 0.6761

Cost time: 74.65303111076355s
